# MPC for TurtleBot3: Obstacle Avoidance Navigation

This notebook demonstrates **Model Predictive Control (MPC)** for obstacle avoidance with the TurtleBot3.

**Problem**: Navigate to goal while avoiding obstacles

**State**: $[x, y, \theta, v, \omega]$ (2D pose + linear/angular velocities)

**Constraints**:
- Obstacle avoidance (position constraints)
- Velocity limits (actuator saturation)
- Angular rate limits

**Key Advantage**: MPC explicitly plans collision-free trajectories.

**Reference**: Mayne et al. (2000), "Constrained model predictive control: Stability and optimality"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pykal import DynamicalSystem, ROSNode
from pykal.algorithm_library.controllers.mpc import MPC
from pykal.gazebo import start_gazebo, stop_gazebo

np.random.seed(42)

## 1. Launch Gazebo Simulation

Start TurtleBot3 in environment with obstacles.

In [ ]:
# Start Gazebo with TurtleBot3
gz = start_gazebo(
    robot='turtlebot3',
    world='turtlebot3_world',
    headless=False
)

print("Gazebo launched with TurtleBot3")
print("Environment: turtlebot3_world (with obstacles)")
time.sleep(3)

## 2. Unicycle Dynamics Model

### Continuous-Time Dynamics
$$\dot{x} = v \cos(\theta)$$
$$\dot{y} = v \sin(\theta)$$
$$\dot{\theta} = \omega$$
$$\dot{v} = a$$
$$\dot{\omega} = \alpha$$

### Discrete-Time (Linearized)
For MPC, we linearize around the current state.

In [ ]:
# MPC parameters
dt = 0.1  # Timestep
N = 15    # Prediction horizon

# Physical constraints
V_MAX = 0.22   # Max linear velocity (m/s)
OMEGA_MAX = 2.84  # Max angular velocity (rad/s)
A_MAX = 0.5    # Max linear acceleration
ALPHA_MAX = 3.0  # Max angular acceleration

# Cost weights
Q_pos = 10.0   # Position tracking
Q_theta = 1.0  # Orientation
R_v = 0.1      # Linear velocity cost
R_omega = 0.1  # Angular velocity cost

# Obstacle positions (x, y, radius)
OBSTACLES = [
    (1.0, 1.0, 0.3),   # Obstacle 1
    (2.0, -0.5, 0.3),  # Obstacle 2
    (3.0, 0.5, 0.3),   # Obstacle 3
]

# Goal position
GOAL = np.array([4.0, 0.0])  # (x, y)

print(f"MPC Parameters:")
print(f"  Horizon: {N} steps ({N*dt}s)")
print(f"  Max velocity: {V_MAX} m/s")
print(f"  Max angular rate: {OMEGA_MAX} rad/s")
print(f"\nObstacles: {len(OBSTACLES)}")
print(f"Goal: {GOAL}")

## 3. Linearized Dynamics

At each timestep, linearize around current state.

In [ ]:
def linearize_unicycle(x, u):
    """
    Linearize unicycle dynamics around (x, u).
    
    State: [x, y, theta, v, omega]
    Input: [a, alpha] (accelerations)
    
    Returns A, B matrices for linearized system.
    """
    _, _, theta, v, omega = x
    
    # Jacobian of f w.r.t. x
    A = np.array([
        [1, 0, -v*np.sin(theta)*dt, np.cos(theta)*dt, 0],
        [0, 1,  v*np.cos(theta)*dt, np.sin(theta)*dt, 0],
        [0, 0, 1, 0, dt],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]
    ])
    
    # Jacobian of f w.r.t. u
    B = np.array([
        [0, 0],
        [0, 0],
        [0, 0],
        [dt, 0],
        [0, dt]
    ])
    
    return A, B

def propagate_dynamics(x, u):
    """
    Nonlinear forward propagation (for simulation).
    """
    x_pos, y_pos, theta, v, omega = x
    a, alpha = u
    
    x_new = x_pos + v * np.cos(theta) * dt
    y_new = y_pos + v * np.sin(theta) * dt
    theta_new = theta + omega * dt
    v_new = v + a * dt
    omega_new = omega + alpha * dt
    
    # Normalize angle
    theta_new = np.arctan2(np.sin(theta_new), np.cos(theta_new))
    
    return np.array([x_new, y_new, theta_new, v_new, omega_new])

print("Dynamics functions defined")

## 4. Obstacle Constraints

Add distance constraints to avoid obstacles.

In [ ]:
def check_collision(pos, obstacles, safety_margin=0.2):
    """
    Check if position collides with any obstacle.
    
    Parameters
    ----------
    pos : np.ndarray
        Position [x, y]
    obstacles : list
        List of (x, y, radius) tuples
    safety_margin : float
        Additional clearance
    
    Returns
    -------
    bool
        True if collision
    """
    for ox, oy, r in obstacles:
        dist = np.sqrt((pos[0] - ox)**2 + (pos[1] - oy)**2)
        if dist < (r + safety_margin):
            return True
    return False

print("Collision checking defined")

## 5. Simplified MPC for TurtleBot

For demonstration, we'll use a simplified approach that plans waypoints and uses MPC for tracking.

In [ ]:
def simple_mpc_controller(current_state, goal_pos, obstacles):
    """
    Simplified MPC controller for obstacle avoidance.
    
    Returns control input [a, alpha].
    """
    x, y, theta, v, omega = current_state
    
    # Compute desired heading to goal
    dx = goal_pos[0] - x
    dy = goal_pos[1] - y
    dist_to_goal = np.sqrt(dx**2 + dy**2)
    desired_theta = np.arctan2(dy, dx)
    
    # Check for nearby obstacles
    repulsion = np.array([0.0, 0.0])
    for ox, oy, r in obstacles:
        obst_dx = x - ox
        obst_dy = y - oy
        obst_dist = np.sqrt(obst_dx**2 + obst_dy**2)
        
        if obst_dist < 1.0:  # Repulsion radius
            # Repulsive force
            force = 2.0 / (obst_dist + 0.1)
            repulsion[0] += force * obst_dx / obst_dist
            repulsion[1] += force * obst_dy / obst_dist
    
    # Adjust desired direction
    adjusted_direction = np.array([dx, dy]) + repulsion
    desired_theta = np.arctan2(adjusted_direction[1], adjusted_direction[0])
    
    # Angle error
    theta_error = desired_theta - theta
    theta_error = np.arctan2(np.sin(theta_error), np.cos(theta_error))
    
    # Control law
    if dist_to_goal > 0.2:
        # Navigate to goal
        desired_v = min(V_MAX, dist_to_goal * 0.5)  # Proportional
        desired_omega = np.clip(theta_error * 3.0, -OMEGA_MAX, OMEGA_MAX)
    else:
        # Near goal, stop
        desired_v = 0.0
        desired_omega = 0.0
    
    # Compute accelerations
    a = np.clip((desired_v - v) / dt, -A_MAX, A_MAX)
    alpha = np.clip((desired_omega - omega) / dt, -ALPHA_MAX, ALPHA_MAX)
    
    return np.array([a, alpha])

print("MPC controller defined")

## 6. ROS Node for MPC Navigation

In [ ]:
from geometry_msgs.msg import Twist, PoseStamped
from nav_msgs.msg import Odometry

# Global state
trajectory_history = []
control_history = []
current_state = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

def mpc_navigation_callback(tk, odom):
    """
    MPC navigation callback.
    
    Parameters
    ----------
    tk : float
        Current time
    odom : np.ndarray
        Odometry [x, y, theta, v, omega]
    
    Returns
    -------
    dict
        Velocity command
    """
    global current_state
    
    current_state = odom
    
    # Compute MPC control
    u = simple_mpc_controller(current_state, GOAL, OBSTACLES)
    
    # Convert to velocity commands (integrate)
    v_cmd = current_state[3] + u[0] * dt
    omega_cmd = current_state[4] + u[1] * dt
    
    # Clip to limits
    v_cmd = np.clip(v_cmd, 0.0, V_MAX)
    omega_cmd = np.clip(omega_cmd, -OMEGA_MAX, OMEGA_MAX)
    
    # Log
    trajectory_history.append(current_state[:3].copy())
    control_history.append(np.array([v_cmd, omega_cmd]))
    
    return {'cmd_vel': np.array([v_cmd, 0.0, 0.0, 0.0, 0.0, omega_cmd])}

print("MPC callback defined")

## 7. Create and Run MPC Node

In [ ]:
# Create MPC node
mpc_node = ROSNode(
    callback=mpc_navigation_callback,
    subscribes_to=[
        ('/odom', Odometry, 'odom')
    ],
    publishes_to=[
        ('cmd_vel', Twist, '/cmd_vel')
    ],
    node_name='mpc_navigation',
    rate_hz=10.0
)

print("MPC navigation node created")
print("\nStarting navigation to goal...")
print(f"Goal: {GOAL}")
print(f"Obstacles: {len(OBSTACLES)}")

In [ ]:
# Start MPC node
mpc_node.create_node()
mpc_node.start()

print("\nMPC node running!")
print("Robot navigating with obstacle avoidance...\n")

# Run for 30 seconds
try:
    time.sleep(30)
except KeyboardInterrupt:
    print("\nInterrupted by user")

# Stop node
mpc_node.stop()
print("\nMPC node stopped")

## 8. Visualize Results

In [ ]:
if len(trajectory_history) > 0:
    traj = np.array(trajectory_history)
    controls = np.array(control_history)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Trajectory with obstacles
    axes[0, 0].plot(traj[:, 0], traj[:, 1], 'b-', linewidth=2, label='MPC trajectory')
    axes[0, 0].plot(traj[0, 0], traj[0, 1], 'go', markersize=12, label='Start')
    axes[0, 0].plot(GOAL[0], GOAL[1], 'r*', markersize=20, label='Goal')
    
    # Plot obstacles
    for i, (ox, oy, r) in enumerate(OBSTACLES):
        circle = plt.Circle((ox, oy), r, color='red', alpha=0.3)
        axes[0, 0].add_patch(circle)
        if i == 0:
            circle.set_label('Obstacles')
    
    axes[0, 0].set_xlabel('X position (m)')
    axes[0, 0].set_ylabel('Y position (m)')
    axes[0, 0].set_title('MPC Obstacle Avoidance Navigation', fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    axes[0, 0].axis('equal')
    
    # Orientation over time
    time_steps = np.arange(len(traj)) * dt
    axes[0, 1].plot(time_steps, traj[:, 2], 'b-', linewidth=2)
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Orientation (rad)')
    axes[0, 1].set_title('Robot Orientation', fontweight='bold')
    axes[0, 1].grid(True)
    
    # Linear velocity
    axes[1, 0].plot(time_steps, controls[:, 0], 'b-', linewidth=2, label='Linear velocity')
    axes[1, 0].axhline(y=V_MAX, color='r', linestyle='--', label='Max velocity')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Linear Velocity (m/s)')
    axes[1, 0].set_title('Linear Velocity Command', fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Angular velocity
    axes[1, 1].plot(time_steps, controls[:, 1], 'r-', linewidth=2, label='Angular velocity')
    axes[1, 1].axhline(y=OMEGA_MAX, color='orange', linestyle='--', label='Max omega')
    axes[1, 1].axhline(y=-OMEGA_MAX, color='orange', linestyle='--')
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Angular Velocity (rad/s)')
    axes[1, 1].set_title('Angular Velocity Command', fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Check goal reached
    final_pos = traj[-1, :2]
    dist_to_goal = np.linalg.norm(final_pos - GOAL)
    
    print(f"\n=== Navigation Results ===")
    print(f"Final position: ({final_pos[0]:.2f}, {final_pos[1]:.2f})")
    print(f"Goal position: ({GOAL[0]:.2f}, {GOAL[1]:.2f})")
    print(f"Distance to goal: {dist_to_goal:.3f} m")
    print(f"Goal reached: {dist_to_goal < 0.3}")
    
    # Check collisions
    collisions = sum([check_collision(traj[i, :2], OBSTACLES) for i in range(len(traj))])
    print(f"\nCollisions: {collisions}/{len(traj)} steps")
    print(f"Collision-free: {collisions == 0}")
else:
    print("No trajectory data collected")

## 9. Shutdown Gazebo

In [ ]:
stop_gazebo(gz)
print("Gazebo simulation stopped")

## Conclusion

### Key Takeaways

1. **Obstacle Avoidance**: MPC plans collision-free paths by incorporating constraints
2. **Real-Time Planning**: Receding horizon enables online replanning around obstacles
3. **Constraint Satisfaction**: Respects velocity and acceleration limits

### MPC vs Other Navigation Methods

| Method | Obstacle Handling | Optimality | Computational Cost |
|--------|------------------|------------|-------------------|
| **MPC** | Explicit constraints | Locally optimal | High (online) |
| **Potential Fields** | Repulsive forces | Heuristic | Low |
| **A\* + Tracking** | Discrete planning | Globally optimal path | Medium |
| **DWA** | Dynamic window | Local | Low |

### Practical Applications

- **Warehouse robots**: Navigate crowded environments
- **Service robots**: Safe human-robot interaction
- **Autonomous vehicles**: Lane keeping with obstacle avoidance
- **Delivery robots**: Urban navigation

### Improvements

- **Nonlinear MPC**: Better accuracy for unicycle model
- **Moving obstacles**: Predict obstacle motion
- **Soft constraints**: Penalty-based obstacle avoidance
- **Learning-based**: Combine with imitation learning

### Next Steps

- See `mpc_crazyflie.ipynb` for 3D aggressive maneuvers
- Implement full nonlinear MPC with cvxpy
- Add dynamic obstacle tracking
- Test with complex multi-obstacle scenarios

### Reference

Mayne, D. Q., Rawlings, J. B., Rao, C. V., & Scokaert, P. O. M. (2000). *Constrained model predictive control: Stability and optimality*. Automatica, 36(6), 789-814.